In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Download dependencies

In [ ]:
!pip install --upgrade transformers

     |████████████████████████████████| 3.1 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 62.9 MB/s 
     |████████████████████████████████| 895 kB 50.5 MB/s 
     |████████████████████████████████| 56 kB 6.4 MB/s 
     |████████████████████████████████| 3.3 MB 61.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install pytorch-transformers

     |████████████████████████████████| 176 kB 5.1 MB/s 
     |████████████████████████████████| 131 kB 40.4 MB/s 
     |████████████████████████████████| 1.2 MB 39.8 MB/s 
     |████████████████████████████████| 8.1 MB 35.4 MB/s 
     |████████████████████████████████| 79 kB 9.9 MB/s 
     |████████████████████████████████| 138 kB 75.4 MB/s 
     |████████████████████████████████| 127 kB 71.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from transformers import __version__
print(__version__)

4.12.2


In [ ]:
!pip install sentencepiece

In [ ]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 5.0 MB/s 
     |████████████████████████████████| 139 kB 75.9 MB/s 
     |████████████████████████████████| 97 kB 7.9 MB/s 
     |████████████████████████████████| 180 kB 71.3 MB/s 
     |████████████████████████████████| 63 kB 2.4 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=c4035bebc158c3431875a9e9b71c0a582431e2e4bf9a015fc32d26aa50a2880f
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=4daa165d13c426f23ee7fd7fcf2f50d656d0ab44e417e89eb0d96f78ebb894a8
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


### Import libraries

In [ ]:
import pandas as pd
import numpy as np
import re
import regex
from tqdm import tqdm

from sklearn import preprocessing, metrics
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve

import torch
import torch.nn.functional as F
from torch import nn, optim
from torch.utils.data import TensorDataset,RandomSampler,SequentialSampler
from torch.utils.data import Dataset, DataLoader

from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

import tensorflow as tf
import transformers
from transformers import XLNetForSequenceClassification, XLNetTokenizer, TFXLNetModel, XLNetModel, AdamW, get_linear_schedule_with_warmup
from transformers import RobertaModel, RobertaTokenizer

import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import WordPunctTokenizer
from string import punctuation
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
print(torch.__version__)

1.9.0+cu111


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

### Load data

In [ ]:
f_path = '/content/drive/MyDrive/3.1/CZ4042 Neural Networks & Deep Learning/Project/emotion/dataset/crowdflower_text_emotion.csv'

In [ ]:
df = pd.read_csv(f_path)

### Data preprocessing and analysis

In [ ]:
df.head()

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


In [ ]:
# Check emotions labels

labels = df.sentiment.unique()
print(labels)

['empty' 'sadness' 'enthusiasm' 'neutral' 'worry' 'surprise' 'love' 'fun'
 'hate' 'happiness' 'boredom' 'relief' 'anger']


In [ ]:
# Remove irrelevant columns
df = df.drop(columns=['tweet_id', 'author'])

In [ ]:
# Check for null cells
df['sentiment'].isnull().values.any()


False

In [ ]:
# Remove stop words, lemmatize, clean text

wordnet_lemmatizer = WordNetLemmatizer()

stop = stopwords.words('english')

for punct in punctuation:
    stop.append(punct)

def clean(x, stop_words):
    word_tokens = WordPunctTokenizer().tokenize(x.lower())
    x = [regex.sub(u'\p{^Latin}', u'', w) for w in word_tokens if w.isalpha() and len(w) > 2]
    return " ".join(x)

df["content_cleaned"] = df.content.apply(lambda x : clean(x, stop)) 

In [ ]:
# Check len of each content
df['content_count'] = df['content_cleaned'].str.split(" ").str.len()

In [ ]:
df.head()

,sentiment,content,content_cleaned,content_count
0,empty,@tiffanylue i know i was listenin to bad habi...,tiffanylue know was listenin bad habit earlier...,12
1,sadness,Layin n bed with a headache ughhhh...waitin o...,layin bed with headache ughhhh waitin your call,8
2,sadness,Funeral ceremony...gloomy friday...,funeral ceremony gloomy friday,4
3,enthusiasm,wants to hang out with friends SOON!,wants hang out with friends soon,6
4,neutral,@dannycastillo We want to trade with someone w...,dannycastillo want trade with someone who has ...,12


### Label encoding classes

In [ ]:
labels_dict = dict(zip(labels, (x for x in range(len(labels)) )))
df['sentiment_val'] = df['sentiment'].map(labels_dict)

In [ ]:
df = df.drop(columns=['sentiment', 'content_count', 'content'])

In [ ]:
df.head()

,content_cleaned,sentiment_val
0,tiffanylue know was listenin bad habit earlier...,0
1,layin bed with headache ughhhh waitin your call,1
2,funeral ceremony gloomy friday,1
3,wants hang out with friends soon,2
4,dannycastillo want trade with someone who has ...,3


### Prep data for training/validation

In [ ]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.content_cleaned
        self.targets = self.data.sentiment_val
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
# Batch Size
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 2
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 2
                }

### BERT

In [ ]:
from transformers import BertTokenizerFast, BertForSequenceClassification, AutoModel, BertConfig, BertModel

#### Large-cased

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-large-cased')
bert = AutoModel.from_pretrained('bert-large-cased')

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
class BERT(nn.Module):

    def __init__(self, config):
        super(BERT, self).__init__()
        self.num_labels = config.num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None,
                position_ids=None, head_mask=None):
      
        # Forward pass through pre-trained BERT
        outputs = self.bert(input_ids, position_ids=position_ids, token_type_ids=token_type_ids,
                            attention_mask=attention_mask, head_mask=head_mask)

        pooler = outputs[-1]
        pooler = self.pre_classifier(pooler
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        
        return outputs

In [ ]:
config = BertConfig(hidden_size=768,
                    hidden_dropout_prob=0.3,
                    num_labels=13)

In [ ]:
# model = BERT(config)

# # push the model to GPU
# model = model.to(device)

### Train/Eval functions

#### Parameters

In [ ]:
LEARNING_RATE = 1e-05
MAX_LEN = 256

# Multi class loss
loss_function = torch.nn.CrossEntropyLoss()

# Accuracy function
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

#### Training

In [ ]:
# For BERT large
def train(model, epoch, training_loader):

  tr_loss = 0
  n_correct = 0
  nb_tr_steps = 0
  nb_tr_examples = 0

  # Tell wandb to watch this model train
  # wandb.watch(model)

  model.train()

  for _, data in tqdm(enumerate(training_loader, 0)):

    ids = data['ids'].to(device, dtype = torch.long)
    mask = data['mask'].to(device, dtype = torch.long)
    token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
    targets = data['targets'].to(device, dtype = torch.long)

    outputs = model(ids, mask, token_type_ids)
    loss = loss_function(outputs, targets)
    tr_loss += loss.item()
    big_val, big_idx = torch.max(outputs.data, dim=1)
    n_correct += calcuate_accuracy(big_idx, targets)

    nb_tr_steps += 1
    nb_tr_examples+=targets.size(0)
    
    if _%5000==0:
        loss_step = tr_loss/nb_tr_steps
        accu_step = (n_correct*100)/nb_tr_examples 

    optimizer.zero_grad()
    loss.backward()
    # Optimizer 
    optimizer.step()

  print('\n')
  epoch_loss = tr_loss/nb_tr_steps
  epoch_accu = (n_correct*100)/nb_tr_examples
  print(f"Training Loss Epoch {epoch+1}: {epoch_loss}")
  print(f"Training Accuracy Epoch {epoch+1}: {epoch_accu} \n")
  
  return epoch_accu, epoch_loss

#### Eval

In [ ]:
def valid(model, testing_loader):

    n_correct = 0; 
    n_wrong = 0; 
    total = 0; 
    tr_loss=0; 
    nb_tr_steps=0; 
    nb_tr_examples=0

    model.eval()

    with torch.no_grad():

        for _, data in tqdm(enumerate(testing_loader, 0)):

            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accuracy(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
        # Reduce LR on val loss plateau
        scheduler.step(loss)

    print('\n')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch : {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu, epoch_loss


### K-Fold (BERT large-cased)

In [ ]:
# Set K-fold
kf = KFold(n_splits=3, shuffle=True, random_state=42)

In [ ]:
# # Setup wandb for logging
# import wandb

# wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
EPOCHS=15
f = 1
df = df.reset_index(drop=True)

for train_index, val_index in kf.split(df):

  # Init model
  model = BERT(config)
  # push the model to GPU
  model = model.to(device)
  # Optimizer
  optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)
  # Reduce LR on plateau
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

  # Init wandb for logging
  # wandb.init(project="Text_Emotion_Recognition_BERT_large_cased3", name=f"3-Fold")

  train2 = df.iloc[train_index]
  test2 = df.iloc[val_index]

  train2 = train2.reset_index(drop=True)
  test2 = df.drop(train2.index).reset_index(drop=True)

  training_set = SentimentData(train2, tokenizer, MAX_LEN)
  testing_set = SentimentData(test2, tokenizer, MAX_LEN)

  training_loader = DataLoader(training_set, **train_params)
  testing_loader = DataLoader(testing_set, **test_params)

  print(f"Training fold {f}...")

  for epoch in range(0, EPOCHS):
    
    # Train
    train_acc, train_loss = train(model, epoch, training_loader)
    # Val
    val_acc, val_loss = valid(model, testing_loader)
    print(f"Epoch {epoch+1} done!")
    # Log metrics
    # wandb.log({"Train_Loss": train_loss, "Val_Loss": val_loss, "Train_Acc": train_acc, "Val_Acc": val_acc, "Epoch": epoch})   

  # wandb.finish()
  f+=1



wandb: Currently logged in as: wasabee (use `wandb login --relogin` to force relogin)


Training fold 1...


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
3334it [12:11,  4.56it/s]



Training Loss Epoch 1: 2.2705960726051466
Training Accuracy Epoch 1: 19.324233105827645 




Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
1667it [01:58, 14.03it/s]

...

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
3334it [12:22,  4.49it/s]



Training Loss Epoch 12: 1.7376379383025087
Training Accuracy Epoch 12: 41.00727518187955 




0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
1667it [02:00, 13.86it/s]




Validation Loss Epoch : 1.7705285341959909
Validation Accuracy Epoch: 42.852857357132145
Epoch 12 done!


0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
1297it [04:48,  4.56it/s]

### Save model

In [ ]:
# Large model
bert_large = model
torch.save(bert_large, '/content/drive/MyDrive/3.1/CZ4042 Neural Networks & Deep Learning/Project/emotion/models/bert_large2/bert_sent_large.bin')
tokenizer.save_vocabulary('/content/drive/MyDrive/3.1/CZ4042 Neural Networks & Deep Learning/Project/emotion/models/bert_large2/')


('/content/drive/MyDrive/3.1/CZ4042 Neural Networks & Deep Learning/Project/emotion/models/bert_large2/vocab.txt',)